In [1]:
pip install pyet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import glob
from pyet.meteo_utils import extraterrestrial_r
from pyet.combination import pm_fao56

In [4]:
# Función para obtener el valor numérico de un string
def get_number(string):
  number = ''
  for i in string:
    for j in range(48,58):
      if chr(j) == i:
        number += i
  return number

# Constantes para obtener Ra
DEG_TO_RAD_CONSTANT = 0.01745329251  
LAT = -33.3971255
lat_rad = float(LAT) *  DEG_TO_RAD_CONSTANT
elevation = 482.0
# Reduciendo reportes a 7 días
horizon_const = 7

# Cantidad de días en que se tiene el "et0 real" respecto a los reportes de la previsión climática a 7 días, a partir del 01-06-2022
n_muestras = 23

# Ruta
RUTA = '/content/drive/MyDrive/Universidad/Evapotranspiration-Prediction-2022/data_processing/'

In [8]:
# Trabajando con los datasets de previsión climática
csv_files = glob.glob(RUTA+'forecast_historical/*report.csv')
csv_files.sort()
csv_files

# 14 days horizon
date = []
t_min = []
t_max = []
temp = []
wind_speed = []
rh = []
rh_min = []
rh_max = []
cod_wc = []

horizon_days = []


for i in range(horizon_const):
 date.append([])
 t_min.append([])
 t_max.append([])
 temp.append([])
 wind_speed.append([])
 rh.append([])
 rh_min.append([])
 rh_max.append([])
 cod_wc.append([])

# Obtención de valores númericos 
for i in range (len(csv_files)):
  ds = pd.read_csv(csv_files[i])
  for h in range (horizon_const):
    t_min[h].append(ds.t_min[h])
    t_max[h].append(ds.t_max[h])
    temp[h].append(ds.temp[h])
    wind_speed[h].append(ds.wind[h])
    rh[h].append(ds.humidity[h])
    rh_min[h].append(ds.humidity_min[h])
    rh_max[h].append(ds.humidity_max[h])
    cod_wc[h].append(1-(ds.clouds[h]/100))
    date[h].append(pd.to_datetime(ds.date[h],format="%Y-%m-%d"))

# Asignando valores numéricos a la condición climática según una aproximación de GUO(2011)
# for i in range (len(weather_condition)):
#   for j in range(len(weather_condition[i])):
#     cond_lluvia = 'lluvia' in weather_condition[i][j].lower()
#     cond_chubascos = 'chubascos' in weather_condition[i][j].lower()
#     cond_nublado = 'nublado' in weather_condition[i][j].lower()
#     cond_soleado = 'soleado' in weather_condition[i][j].lower()
#     if (cond_lluvia):
#       if (('tarde' or 'mañana') in weather_condition[i][j].lower()):
#         cod_wc[i].append(0.1)
#       else:
#         cod_wc[i].append(0.05)
#     elif (cond_chubascos):
#       cod_wc[i].append(0.2)
  
#     elif (cond_nublado):
#       if ('parcialmente' in weather_condition[i][j].lower()):
#         cod_wc[i].append(0.7)
#       elif ('muy' in weather_condition[i][j].lower()):
#         cod_wc[i].append(0.3)
#       else:
#         cod_wc[i].append(0.5)

#     elif (cond_soleado):
#       if ('mayormente' in weather_condition[i][j].lower()):
#         cod_wc[i].append(0.7)
#       else:
#         cod_wc[i].append(0.9)

#     else:
#       cod_wc[i].append(0.5)


# Creación de DataFrames para los 14 días de pronóstico # Indexando Fecha y añadiendo Ra complementaria y ET0 calculado por PMFAO56 con datos de previsión climática
for i in range (horizon_const):
  horizon_days.append(pd.DataFrame({'date':date[i],'tmin':t_min[i],'tmax':t_max[i],'temp':temp[i],'rh':rh[i],'rhmin':rh_min[i],'rhmax':rh_max[i],'wind':wind_speed[i],'cod_wc':cod_wc[i]}))
  horizon_days[i].date = pd.to_datetime(horizon_days[i].date,format='%Y-%m-%d')
  horizon_days[i] = horizon_days[i].set_index('date')
  horizon_days[i] = horizon_days[i].sort_values('date')
  horizon_days[i]['ra'] = extraterrestrial_r(horizon_days[i].index,lat_rad)
  horizon_days[i]['et0_pmfao56_raxwc'] = pm_fao56(tmean=horizon_days[i].temp,wind = horizon_days[i].wind,rs = (horizon_days[i].ra*horizon_days[i].cod_wc),rh=horizon_days[i].rh,tmax = horizon_days[i].tmax,tmin = horizon_days[i].tmin,lat = lat_rad,elevation=elevation)
  # horizon_days[i] = horizon_days[i].iloc[0:n_muestras+1]

In [13]:
horizon_days[6]

,tmin,tmax,temp,rh,rhmin,rhmax,wind,cod_wc,ra,et0_pmfao56_raxwc
date,,,,,,,,,,
2021-01-07,8.89,31.27,18.899167,50.365833,20.50,78.95,2.663333,0.998750,43.856060,8.792655
2021-01-08,8.63,30.83,18.072187,56.093021,18.20,89.63,2.898333,0.999896,43.800098,8.457304
2021-01-09,6.97,31.02,17.227708,53.662083,12.42,92.02,2.526875,0.989375,43.740775,8.301622
2021-01-10,5.48,31.37,16.980313,49.005417,10.69,86.26,2.373958,0.996979,43.678089,8.480470
2021-01-11,7.22,31.45,18.057604,48.418542,15.90,82.98,2.496875,1.000000,43.612041,8.700435
...,...,...,...,...,...,...,...,...,...,...
2022-06-03,5.01,14.23,9.164271,72.654479,43.08,92.07,0.997083,0.193229,17.240746,0.853766
2022-06-04,7.66,13.68,9.787187,80.224479,55.60,93.70,1.432292,0.006563,17.162499,0.377787
2022-06-05,7.50,16.97,11.186250,73.161458,44.86,95.60,1.553333,0.162083,17.088531,1.019395


In [14]:
for i in range(len(horizon_days)):
  horizon_days[i].to_csv(RUTA+'processed_forecast_historical/'+str(i+1)+'-Day(s)-forecast-Horizon.csv')